In [19]:
import jax
import jax.numpy as jnp
import numpy as np
import pylab as plt
import scipy
import os
os.environ['JAX_PLATFORM_NAME'] = 'cpu' 

In [4]:
import pandas as pd

# Read in the songsDataset.csv file
songs_df = pd.read_csv('songsDataset.csv')

# Print the DataFrame
print(songs_df)


         'userID'  'songID'  'rating'
0               0      7171         5
1               0      8637         4
2               0     21966         4
3               0     35821         5
4               0     82446         5
...           ...       ...       ...
1999995    199999     75840         2
1999996    199999     85921         1
1999997    199999    116763         1
1999998    199999    128758         1
1999999    199999    129321         1

[2000000 rows x 3 columns]


In [31]:
# prompt: calculate the number of unique songID and userID appear in the df seperately

print("Number of unique songID:", songs_df["'songID'"].nunique())
print("Number of unique userID:", songs_df["'userID'"].nunique())
largest_song_id = songs_df["'songID'"].max()
smallest_song_id = songs_df["'songID'"].min()
print("Largest songID:", largest_song_id)
print("Smallest songID:", smallest_song_id)

Number of unique songID: 127771
Number of unique userID: 200000
Largest songID: 136735
Smallest songID: 0


In [6]:
N_songs = 127771
N_users = 200000
N_ratings = 2000000

In [38]:
# dimension of the factos
d_factors = 5

# create factors for each users and each film
U_star = np.random.normal(0,1, size=(N_users, d_factors)) / np.sqrt(d_factors)
S_star = np.random.normal(0,1, size=(largest_song_id+1, d_factors)) / np.sqrt(d_factors)

# create ratings
Obs_users = songs_df["'userID'"].values
Obs_songs = songs_df["'songID'"].values
Ratings = songs_df["'rating'"].values

print(Obs_users[0])

0


In [39]:
# DATA
Obs_users.shape, Obs_songs.shape, Ratings.shape

((2000000,), (2000000,), (2000000,))

In [40]:
# our parameters
# all the factor-vectors for the users: dimension N_users x d_factors
U_init = np.random.normal(0,1, size=(N_users, d_factors)) / np.sqrt(d_factors)
# all the factor-vectors for the films: dimension N_films x d_factors
S_init = np.random.normal(0,1, size=(largest_song_id+1, d_factors)) / np.sqrt(d_factors)

In [34]:
@jax.jit
def loss(U_single, S_single, rating_single):
    """ compute the error for a single pair of user and film
    """
    pred = jnp.dot(U_single, S_single)
    error = rating_single - pred
    return error**2

loss_grad = jax.jit( jax.grad(loss, argnums=(0,1)) )

In [35]:
def mse(U_batch, S_batch, ratings_batch):
    """ compute the MSE on a batch of users x films"""
    predictions = np.sum(U_batch * S_batch, axis=1)
    errors = ratings_batch - predictions
    return np.mean(errors**2)

In [43]:
niter = 500
learning_rate = 0.1

U = U_init.copy()
S = S_init.copy()

for it in range(niter):
    for index in range(N_ratings):
        # fetch a random user, film and rating
        #index = np.random.randint(N_ratings)
        user_id = Obs_users[index]
        song_id = Obs_songs[index]
        rating = Ratings[index]
        # lt's compute the gradient
        U_grad, S_grad = loss_grad(U[user_id], S[song_id], rating)
        # let's do the update
        U[user_id] -= learning_rate * U_grad
        S[song_id] -= learning_rate * S_grad
    # let's computhe the MSE
    # print every 10 iterations
    #if it % 10 == 0:
    U_batch = U[Obs_users[:10000]]
    S_batch = S[Obs_songs[:10000]]
    ratings_batch = Ratings[:10000]
    mse_current = mse(U_batch, S_batch, ratings_batch)
    print(f"iter {it}, mse = {mse_current:.5f}")

iter 0, mse = nan
iter 1, mse = nan
iter 2, mse = nan


KeyboardInterrupt: 